# Line Search SQP Algorithm

Implemented from Algorithm 18.3 from Nocedal (2008), p. 545.

## Problem formulation

We attempt to solve the general nonlinear programming problem (`18.10`)
 
$$ \min f(x) $$

subject to
$$ 
\begin{align*}
    c_i(x) &= 0, \quad i \in \mathcal{E}, \\
    c_i(x) &\geq 0, \quad i \in \mathcal{I}. \\
\end{align*}
$$

To model this problem we now linearize both the inequality and equality
constraints to obtain (`18.11`)

$$ 
    \min_p f_k + \nabla f_k^T p
    + \frac{1}{2} p^T \nabla_{xx}^2 \mathcal{L}_k p
$$

subject to
$$ 
\begin{align*}
    \nabla c_i(x_k)^T p + c_i(x_k) &= 0, \quad i \in \mathcal{E}, \\
    \nabla c_i(x_k)^T p + c_i(x_k) &\geq 0, \quad i \in \mathcal{I}. \\
\end{align*}
$$

We can use one of the algorithms for quadratic programming to solve this
problem. The new iterate is given by $(x_k + p_k, \lambda_{k+1})$ where $p_k$
and $\lambda_{k+1}$ are the solution and the corresponding Lagrange multiplier 
of (`18.11`)

In [1]:
import numpy as np
from optimus import ls_sqp
from scipy.optimize import rosen, rosen_der, minimize, LinearConstraint

Let's try an example with the [Rosenbrock function](https://en.wikipedia.org/wiki/Rosenbrock_function):

$$ f(x_1, x_2) = (a - x_1)^2 + b(x_2 - x_1^2)^2 $$

with global minumum at $(a,a^2)$.

In [2]:
A = np.array([
    [ 1, -2],
    [-1, -1],
    [-1,  2],
    [ 1,  0],
    [ 0,  1]
])
b = np.array([-2,-6,-2,0,0])

In [3]:
def fun(x):
    return rosen(x), rosen_der(x)

def restr(x):
    return np.dot(A, x) - b, A

x0 = np.array([2.,0.])

## Example using SciPy

In [4]:
constr = LinearConstraint(A, b, [np.inf]*5)
res = minimize(fun=fun, x0=x0, method='trust-constr', jac=True, constraints=constr)
res.x # should be (1., 1.)

array([1.00000014, 1.00000029])

In [5]:
np.dot(A, res.x) >= b

array([ True,  True,  True,  True,  True])

In [6]:
x, lam = ls_sqp(fun, restr, x_0=x0, lam_0=np.ones(5), B_0=np.eye(x0.size), eta=0.4, tau=0.7, maxiters=1000, tol=10e-10)

iter 1:
 - x: [2.02027009 0.00895708]
 - l: [ 1.50121285 -6.96578101 -5.17930732  1.49120926  2.6230299 ]
 - ||kkt||: 3376.7218805721777
 - alpha_k: 0.004747561509942996

iter 2:
 - x: [1.66175999 1.83088   ]
 - l: [1.56732982e+02 1.52500231e-07 9.55932453e-08 2.30101207e-07
 2.08846555e-07]
 - ||kkt||: 480.33208960155343
 - alpha_k: 1

iter 3:
 - x: [1.46316648 1.73158324]
 - l: [2.31938167e+01 1.72257187e-08 1.20806130e-08 3.30259426e-08
 2.79065141e-08]
 - ||kkt||: 220.14208683892826
 - alpha_k: 1

iter 4:
 - x: [1.3420919  1.67104595]
 - l: [9.14610356e+00 7.09220785e-09 5.29586180e-09 1.57839021e-08
 1.26767592e-08]
 - ||kkt||: 61.9011807187666
 - alpha_k: 1

iter 5:
 - x: [1.29302274 1.64651137]
 - l: [1.70476177e+00 1.36663059e-09 1.04563158e-09 3.23468891e-09
 2.54023530e-09]
 - ||kkt||: 12.13200399849888
 - alpha_k: 1

iter 6:
 - x: [1.28118847 1.64059423]
 - l: [1.28915172e-02 7.65342905e-12 5.88972942e-12 1.83883310e-11
 1.43599905e-11]
 - ||kkt||: 0.9954092913528495
 - alph

In [7]:
x

array([1., 1.])

In [8]:
np.dot(A, x) >= b

array([ True,  True,  True,  True,  True])